In [6]:
import numpy as np 
from dash_website.

time_series = np.load("/home/theovincent/HMS/Dash-Website/all_data/page15_AttentionMapsTimeSeries/img/Age/Heart/ECG/TimeSeries/Female/middle/normal/Saliency_Age_Heart_ECG_TimeSeries_Female_middle_normal_0.npy")
channel = 0

In [5]:
time_series[channel]

array([-35.248497 , -34.231373 , -27.214247 , ...,   2.4926002,
        -1.4652108,   1.5769782], dtype=float32)

In [10]:
import plotly.graph_objs as go 

go.Figure(go.Scatter(y=time_series[channel], mode="markers", marker=dict(size=5)))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'size': 5},
              'mode': 'markers',
              'type': 'scatter',
              'y': array([-35.248497 , -34.231373 , -27.214247 , ...,   2.4926002,  -1.4652108,
                            1.5769782], dtype=float32)}],
    'layout': {'template': '...'}
})